In [1]:
import pandas as pd
import os
import xml.dom.minidom
import xml.etree.cElementTree as et
import numpy as np

## GeoNames POI Extraction

In [2]:
dir_geonames = os.path.dirname(os.getcwd())+'\\data\\gazetteers\\allCountries.txt'

In [3]:
df_geonames = pd.read_csv(dir_geonames,sep='\t',names=('geonameid','name','asciiname','alternatenames','latitude',
                                                        'longitude','feature class','feature code','country code','cc2',
                                                        'admin1 code','admin2 code','admin3 code','admin4 code','population',
                                                        'elevation','dem','timezone','modification date'))
df_geonames.head()

C:\Users\Zilong Liu\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (8,9,10,11,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,admin1 code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification date
0,2986043,Pic de Font Blanca,Pic de Font Blanca,"Pic de Font Blanca,Pic du Port",42.64991,1.53335,T,PK,AD,NaN,00,NaN,NaN,NaN,0,NaN,2860,Europe/Andorra,2014-11-05
1,2994701,Roc Mélé,Roc Mele,"Roc Mele,Roc Meler,Roc Mélé",42.58765,1.74028,T,MT,AD,"AD,FR",00,NaN,NaN,NaN,0,NaN,2803,Europe/Andorra,2020-06-10
2,3007683,Pic des Langounelles,Pic des Langounelles,Pic des Langounelles,42.61203,1.47364,T,PK,AD,"AD,FR",00,NaN,NaN,NaN,0,NaN,2685,Europe/Andorra,2014-11-05
3,3017832,Pic de les Abelletes,Pic de les Abelletes,"Pic de la Font-Negre,Pic de la Font-Nègre,Pic ...",42.52535,1.73343,T,PK,AD,FR,A9,66,663.0,66146.0,0,NaN,2411,Europe/Andorra,2014-11-05
4,3017833,Estany de les Abelletes,Estany de les Abelletes,"Estany de les Abelletes,Etang de Font-Negre,Ét...",42.52915,1.73362,H,LK,AD,FR,A9,NaN,NaN,NaN,0,NaN,2260,Europe/Andorra,2014-11-05


In [4]:
df_geonames = df_geonames[['geonameid','name','asciiname','latitude','longitude']]
df_geonames.head()

,geonameid,name,asciiname,latitude,longitude
0,2986043,Pic de Font Blanca,Pic de Font Blanca,42.64991,1.53335
1,2994701,Roc Mélé,Roc Mele,42.58765,1.74028
2,3007683,Pic des Langounelles,Pic des Langounelles,42.61203,1.47364
3,3017832,Pic de les Abelletes,Pic de les Abelletes,42.52535,1.73343
4,3017833,Estany de les Abelletes,Estany de les Abelletes,42.52915,1.73362


In [5]:
df_geonames.to_csv(os.path.dirname(os.getcwd())+'\\data\\extracted-annotated-locations\\GeoNames_poi.csv')

## GeoWiki POI Extraction

In [6]:
dir_geowiki = os.path.dirname(os.getcwd())+'\\data\\training-corpora\\geowiki.txt'

In [7]:
num = 0
geowiki_output_file = open(os.path.dirname(os.getcwd())+'\\data\\extracted-annotated-locations\\GeoWiki_poi.csv','w',encoding="UTF-8")
geowiki_output_file.write('Place\tLatitude\tLongitude\n')
for line in open(dir_geowiki,'r',encoding="UTF-8"):
    num += 1
    if (num % 2 == 0):
        line = line.strip('\n')
        line = line.split('\t')
        geowiki_output_file.write(line[1]+'\t'+line[2]+'\t'+line[3]+'\n')
geowiki_output_file.close()

## Define MLG Key Calculation Function

In [8]:
def calc_mlg_key(toponym, old_coord):
    toponym = toponym.strip(' ')
    old_coord_split = old_coord.split(',')
    old_lat = old_coord_split[0].strip(' ')
    old_lon = old_coord_split[1].strip(' ')
    return toponym+' '+old_lat+','+old_lon

## LGL POI Extraction

In [9]:
xml_lgl = os.path.dirname(os.getcwd())+'\\data\\evaluation-corpora\\original-datasets\\lgl.xml'
parsed_xml_lgl = et.parse(xml_lgl)

In [10]:
url_list = []
text_list = []
toponyms_list = []

for article in parsed_xml_lgl.getroot():
    url = article.find('url')
    text = article.find('text')
    toponyms = article.find('toponyms')
    toponym_list = []
    for toponym in toponyms:
        start = toponym.find('start')
        end = toponym.find('end')
        gaztag = toponym.find('gaztag')
        try:
            name = gaztag.find('name')
            lat = gaztag.find('lat')
            lon = gaztag.find('lon')
            toponym_list.append({'name':name.text, 'start':start.text, 'end': end.text, 'lat': lat.text, 'lon': lon.text})    
        except AttributeError:
            name = toponym.find('phrase')
            toponym_list.append({'name':name.text, 'start':start.text, 'end': end.text, 'lat': None, 'lon': None})    
    
    url_list.append(url.text)
    text_list.append(text.text)
    toponyms_list.append(toponym_list)
    
df_lgl = pd.DataFrame({'url' :url.text, 'text': text_list, 'toponyms': toponyms_list})
df_lgl.head()

,url,text,toponyms
0,http://www.concordmonitor.com/apps/pbcs.dll/ar...,Alexandria woman charged in connection with Ke...,"[{'name': 'Alexandria', 'start': '0', 'end': '..."
1,http://www.concordmonitor.com/apps/pbcs.dll/ar...,Avoyelles task force arrests 14. The Task Forc...,"[{'name': 'Avoyelles Parish', 'start': '0', 'e..."
2,http://www.concordmonitor.com/apps/pbcs.dll/ar...,Alexandria parents charged with hitting their ...,"[{'name': 'Alexandria', 'start': '0', 'end': '..."
3,http://www.concordmonitor.com/apps/pbcs.dll/ar...,Deputies arrest Pointe Coupee teen on weapon c...,"[{'name': 'Pointe Coupee', 'start': '16', 'end..."
4,http://www.concordmonitor.com/apps/pbcs.dll/ar...,Water boil advisory issued in downtown Pinevil...,"[{'name': 'Pineville', 'start': '39', 'end': '..."


In [11]:
name_list = []
lat_list = []
lon_list = []

for article in parsed_xml_lgl.getroot():
    toponyms = article.find('toponyms')
    for toponym in toponyms:
        gaztag = toponym.find('gaztag')
        try:
            name = gaztag.find('name')
            lat = gaztag.find('lat')
            lon = gaztag.find('lon')
            name_list.append(name.text)
            lat_list.append(lat.text)
            lon_list.append(lon.text)
        except AttributeError:
            name = toponym.find('phrase')
            name_list.append(name.text)
            lat_list.append(None)
            lon_list.append(None)

df_lgl_poi = pd.DataFrame({'name' :name_list, 'lat': lat_list, 'lon': lon_list})
df_lgl_poi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5088 entries, 0 to 5087
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    5088 non-null   object
 1   lat     4462 non-null   object
 2   lon     4462 non-null   object
dtypes: object(3)
memory usage: 119.4+ KB


In [12]:
df_lgl_poi.head()

,name,lat,lon
0,Alexandria,31.3113,-92.4451
1,Alexandria,31.3113,-92.4451
2,Rapides Parish,31.1669,-92.4835
3,Avoyelles Parish,31.0669,-91.9668
4,Cottonport,30.9841,-92.0535


In [13]:
df_lgl_poi = df_lgl_poi.drop_duplicates()
df_lgl_poi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1571 entries, 0 to 5081
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    1571 non-null   object
 1   lat     1086 non-null   object
 2   lon     1086 non-null   object
dtypes: object(3)
memory usage: 49.1+ KB


In [14]:
df_lgl_poi.head()

,name,lat,lon
0,Alexandria,31.3113,-92.4451
2,Rapides Parish,31.1669,-92.4835
3,Avoyelles Parish,31.0669,-91.9668
4,Cottonport,30.9841,-92.0535
8,Memphis St.,None,None


In [15]:
mlg_lgl = pd.read_csv(os.path.dirname(os.getcwd())+'\\data\\evaluation-corpora\\data-patches\\lgl_patches.tsv', sep = '\t', header = None)
mlg_lgl = mlg_lgl.rename(columns = {0:'toponym', 1:'old_coord', 2:'new_coord'})
mlg_lgl.head()

,toponym,old_coord,new_coord
0,Africa,"7.1881 , 21.0938","1.0 , 17.0"
1,Akron,"41.0814 , -81.519","41.073055555556 , -81.517777777778"
2,Akuressa,"6.09639 , 80.4808","6.09638889 , 80.48083333"
3,Alabama,"32.7504 , -86.7503","32.7 , -86.7"
4,Alachua County,"29.7086 , -82.3332","29.683333333333 , -82.366666666667"


In [16]:
mlg_lgl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 901 entries, 0 to 900
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   toponym    901 non-null    object
 1   old_coord  901 non-null    object
 2   new_coord  901 non-null    object
dtypes: object(3)
memory usage: 21.2+ KB


In [17]:
mlg_lgl['mlg_key'] = mlg_lgl.apply(lambda x: calc_mlg_key(x['toponym'],x['old_coord']),axis = 1)
mlg_lgl.head()

,toponym,old_coord,new_coord,mlg_key
0,Africa,"7.1881 , 21.0938","1.0 , 17.0","Africa 7.1881,21.0938"
1,Akron,"41.0814 , -81.519","41.073055555556 , -81.517777777778","Akron 41.0814,-81.519"
2,Akuressa,"6.09639 , 80.4808","6.09638889 , 80.48083333","Akuressa 6.09639,80.4808"
3,Alabama,"32.7504 , -86.7503","32.7 , -86.7","Alabama 32.7504,-86.7503"
4,Alachua County,"29.7086 , -82.3332","29.683333333333 , -82.366666666667","Alachua County 29.7086,-82.3332"


In [18]:
def calc_lgl_key(name, lat, lon):
    name = name.strip(' ')
    if (lat != None) & (lon != None):
        lat = lat.strip(' ')
        lon = lon.strip(' ')
        return name+' '+lat+','+lon

In [19]:
df_lgl_poi['lgl_key'] = df_lgl_poi.apply(lambda x: calc_lgl_key(x['name'], x['lat'], x['lon']),axis = 1)
df_lgl_poi.head()

,name,lat,lon,lgl_key
0,Alexandria,31.3113,-92.4451,"Alexandria 31.3113,-92.4451"
2,Rapides Parish,31.1669,-92.4835,"Rapides Parish 31.1669,-92.4835"
3,Avoyelles Parish,31.0669,-91.9668,"Avoyelles Parish 31.0669,-91.9668"
4,Cottonport,30.9841,-92.0535,"Cottonport 30.9841,-92.0535"
8,Memphis St.,None,None,None


In [20]:
df_lgl_poi_unified = pd.merge(df_lgl_poi, mlg_lgl, how = 'outer', left_on = 'lgl_key', right_on = 'mlg_key')
df_lgl_poi_unified.head()

,name,lat,lon,lgl_key,toponym,old_coord,new_coord,mlg_key
0,Alexandria,31.3113,-92.4451,"Alexandria 31.3113,-92.4451",Alexandria,"31.3113 , -92.4451","31.2928 , -92.4592","Alexandria 31.3113,-92.4451"
1,Rapides Parish,31.1669,-92.4835,"Rapides Parish 31.1669,-92.4835",Rapides Parish,"31.1669 , -92.4835","31.2 , -92.54","Rapides Parish 31.1669,-92.4835"
2,Avoyelles Parish,31.0669,-91.9668,"Avoyelles Parish 31.0669,-91.9668",Avoyelles Parish,"31.0669 , -91.9668","31.07 , -92.0","Avoyelles Parish 31.0669,-91.9668"
3,Cottonport,30.9841,-92.0535,"Cottonport 30.9841,-92.0535",Cottonport,"30.9841 , -92.0535","30.98408 , -92.05346","Cottonport 30.9841,-92.0535"
4,Memphis St.,None,None,None,NaN,NaN,NaN,NaN


In [21]:
df_lgl_poi_unified.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1571 entries, 0 to 1570
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       1571 non-null   object
 1   lat        1086 non-null   object
 2   lon        1086 non-null   object
 3   lgl_key    1086 non-null   object
 4   toponym    901 non-null    object
 5   old_coord  901 non-null    object
 6   new_coord  901 non-null    object
 7   mlg_key    901 non-null    object
dtypes: object(8)
memory usage: 110.5+ KB


In [22]:
df_lgl_poi_unified_filtered = df_lgl_poi_unified.dropna(subset=['new_coord'])
df_lgl_poi_unified_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 901 entries, 0 to 1569
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       901 non-null    object
 1   lat        901 non-null    object
 2   lon        901 non-null    object
 3   lgl_key    901 non-null    object
 4   toponym    901 non-null    object
 5   old_coord  901 non-null    object
 6   new_coord  901 non-null    object
 7   mlg_key    901 non-null    object
dtypes: object(8)
memory usage: 63.4+ KB


In [23]:
df_lgl_poi_unified_filtered.to_csv(os.path.dirname(os.getcwd())+'\\data\\extracted-annotated-locations\\LGL_poi.csv')

## GeoVirus POI Extraction

In [24]:
xml_geovirus = os.path.dirname(os.getcwd())+'\\data\\evaluation-corpora\\original-datasets\\GeoVirus.xml'
parsed_xml_geovirus = et.parse(xml_geovirus)

In [25]:
source_list = []
text_list = []
locations_list = []

for article in parsed_xml_geovirus.getroot():
    source = article.find('source')
    text = article.find('text')
    locations = article.find('locations')
    location_list = []
    for location in locations:
        name = location.find('name')
        start = location.find('start')
        end = location.find('end')
        lat = location.find('lat')
        lon = location.find('lon')
        page = location.find('page')
        
        page_split = page.text.split('/')
        wikipedia_name = page_split[len(page_split)-1]
        wikipedia_name = wikipedia_name.replace("_"," ")
        wikipedia_name = wikipedia_name.replace("%27","\'")
        
        location_list.append({'name':name.text, 'wikipedia_name':wikipedia_name, 'start':start.text, 'end': end.text, 'lat': lat.text, 'lon': lon.text, 'page': page.text})    
    
    source_list.append(source.text)
    text_list.append(text.text)
    locations_list.append(location_list)
    

df_geovirus = pd.DataFrame({'source' :source_list, 'text': text_list, 'locations': locations_list})
df_geovirus.head()

,source,text,locations
0,https://en.wikinews.org/wiki/Philippines_is_cu...,"The Philippine Department of Agriculture (DA),...","[{'name': 'Pandi', 'wikipedia_name': 'Pandi, B..."
1,https://en.wikinews.org/wiki/Philippines_is_cu...,The agriculture officials earlier announced th...,"[{'name': 'Bulacan', 'wikipedia_name': 'Bulaca..."
2,https://en.wikinews.org/wiki/Philippines_is_cu...,Local police has also arrested four vendors an...,"[{'name': 'Balintawak', 'wikipedia_name': 'Bal..."
3,https://en.wikinews.org/wiki/US_soldier_who_di...,Tests yesterday proved negative for Ebola on t...,"[{'name': 'Texas', 'wikipedia_name': 'Texas', ..."
4,https://en.wikinews.org/wiki/Patient_under_eva...,A patient was admitted to a hospital in Washin...,"[{'name': 'Washington', 'wikipedia_name': 'Was..."


In [26]:
name_list = []
lat_list = []
lon_list = []
page_list = []
wikipedia_name_list = []

for article in parsed_xml_geovirus.getroot():
    locations = article.find('locations')
    for location in locations:
        name = location.find('name')
        lat = location.find('lat')
        lon = location.find('lon')
        page = location.find('page')
        
        page_split = page.text.split('/')
        wikipedia_name = page_split[len(page_split)-1]
        wikipedia_name = wikipedia_name.replace("_"," ")
        wikipedia_name = wikipedia_name.replace("%27","\'")
        
        name_list.append(name.text)
        lat_list.append(lat.text)
        lon_list.append(lon.text)
        page_list.append(page.text)
        wikipedia_name_list.append(wikipedia_name)

df_geovirus_poi = pd.DataFrame({'name' :name_list, 'wikipedia_name':wikipedia_name_list, 'lat': lat_list, 'lon': lon_list, 'page': page_list})
df_geovirus_poi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2167 entries, 0 to 2166
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            2167 non-null   object
 1   wikipedia_name  2167 non-null   object
 2   lat             2167 non-null   object
 3   lon             2167 non-null   object
 4   page            2167 non-null   object
dtypes: object(5)
memory usage: 84.8+ KB


In [27]:
df_geovirus_poi.head()

,name,wikipedia_name,lat,lon,page
0,Pandi,"Pandi, Bulacan",14.87,120.95,"https://en.wikipedia.org/wiki/Pandi,_Bulacan"
1,Central Luzon,Central Luzon,15.47,120.75,https://en.wikipedia.org/wiki/Central_Luzon
2,Bulacan,Bulacan,15,121.08,https://en.wikipedia.org/wiki/Bulacan
3,Bulacan,Bulacan,15,121.08,https://en.wikipedia.org/wiki/Bulacan
4,Manaoag,"Manaoag, Pangasinan",16.04,120.48,"https://en.wikipedia.org/wiki/Manaoag,_Pangasinan"


In [28]:
df_geovirus_poi = df_geovirus_poi.drop_duplicates()
df_geovirus_poi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 700 entries, 0 to 2154
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            700 non-null    object
 1   wikipedia_name  700 non-null    object
 2   lat             700 non-null    object
 3   lon             700 non-null    object
 4   page            700 non-null    object
dtypes: object(5)
memory usage: 32.8+ KB


In [29]:
df_geovirus_poi.head()

,name,wikipedia_name,lat,lon,page
0,Pandi,"Pandi, Bulacan",14.87,120.95,"https://en.wikipedia.org/wiki/Pandi,_Bulacan"
1,Central Luzon,Central Luzon,15.47,120.75,https://en.wikipedia.org/wiki/Central_Luzon
2,Bulacan,Bulacan,15,121.08,https://en.wikipedia.org/wiki/Bulacan
4,Manaoag,"Manaoag, Pangasinan",16.04,120.48,"https://en.wikipedia.org/wiki/Manaoag,_Pangasinan"
5,Pangasinan,Pangasinan,15.92,120.33,https://en.wikipedia.org/wiki/Pangasinan


In [30]:
mlg_geovirus = pd.read_csv(os.path.dirname(os.getcwd())+'\\data\\evaluation-corpora\\data-patches\\GeoVirus_patches.tsv', sep = '\t', header = None)
mlg_geovirus = mlg_geovirus.rename(columns = {0:'toponym',1: 'old_coord',2: 'new_coord'})
mlg_geovirus.head()

,toponym,old_coord,new_coord
0,"Abilene, Texas","32.45 , -99.75","32.446388888889 , -99.745555555556"
1,Acapulco,"16.86 , -99.88","16.8628695 , -99.8870087"
2,Aceh,"5.55 , 95.31","4.3 , 96.9"
3,Adhamiyah,"33.36 , 44.36","33.36841667 , 44.36213889"
4,Afghanistan,"33 , 65","34.0 , 66.0"


In [31]:
mlg_geovirus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 623 entries, 0 to 622
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   toponym    623 non-null    object
 1   old_coord  623 non-null    object
 2   new_coord  623 non-null    object
dtypes: object(3)
memory usage: 14.7+ KB


In [32]:
mlg_geovirus['mlg_key'] = mlg_geovirus.apply(lambda x: calc_mlg_key(x['toponym'],x['old_coord']),axis = 1)
mlg_geovirus.head()

,toponym,old_coord,new_coord,mlg_key
0,"Abilene, Texas","32.45 , -99.75","32.446388888889 , -99.745555555556","Abilene, Texas 32.45,-99.75"
1,Acapulco,"16.86 , -99.88","16.8628695 , -99.8870087","Acapulco 16.86,-99.88"
2,Aceh,"5.55 , 95.31","4.3 , 96.9","Aceh 5.55,95.31"
3,Adhamiyah,"33.36 , 44.36","33.36841667 , 44.36213889","Adhamiyah 33.36,44.36"
4,Afghanistan,"33 , 65","34.0 , 66.0","Afghanistan 33,65"


In [33]:
def calc_geovirus_key(wikipedia_name, lat, lon):
    wikipedia_name = wikipedia_name.strip(' ')
    if (lat != None) & (lon != None):
        lat = lat.strip(' ')
        lon = lon.strip(' ')
        return wikipedia_name+' '+lat+','+lon

In [34]:
df_geovirus_poi['geovirus_key'] = df_geovirus_poi.apply(lambda x: calc_geovirus_key(x['wikipedia_name'], x['lat'], x['lon']),axis = 1)
df_geovirus_poi.head()

,name,wikipedia_name,lat,lon,page,geovirus_key
0,Pandi,"Pandi, Bulacan",14.87,120.95,"https://en.wikipedia.org/wiki/Pandi,_Bulacan","Pandi, Bulacan 14.87,120.95"
1,Central Luzon,Central Luzon,15.47,120.75,https://en.wikipedia.org/wiki/Central_Luzon,"Central Luzon 15.47,120.75"
2,Bulacan,Bulacan,15,121.08,https://en.wikipedia.org/wiki/Bulacan,"Bulacan 15,121.08"
4,Manaoag,"Manaoag, Pangasinan",16.04,120.48,"https://en.wikipedia.org/wiki/Manaoag,_Pangasinan","Manaoag, Pangasinan 16.04,120.48"
5,Pangasinan,Pangasinan,15.92,120.33,https://en.wikipedia.org/wiki/Pangasinan,"Pangasinan 15.92,120.33"


In [35]:
df_geovirus_poi_unified = pd.merge(df_geovirus_poi, mlg_geovirus, how = 'outer', left_on = 'geovirus_key', right_on = 'mlg_key')
df_geovirus_poi_unified.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 700 entries, 0 to 699
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            700 non-null    object
 1   wikipedia_name  700 non-null    object
 2   lat             700 non-null    object
 3   lon             700 non-null    object
 4   page            700 non-null    object
 5   geovirus_key    700 non-null    object
 6   toponym         664 non-null    object
 7   old_coord       664 non-null    object
 8   new_coord       664 non-null    object
 9   mlg_key         664 non-null    object
dtypes: object(10)
memory usage: 60.2+ KB


In [36]:
df_geovirus_poi_unified.to_csv(os.path.dirname(os.getcwd())+'\\data\\extracted-annotated-locations\\GeoVirus_poi.csv')

## WikToR POI Extraction

In [37]:
xml_wiktor = os.path.dirname(os.getcwd())+'\\data\\evaluation-corpora\\original-datasets\\WikToR.xml'
parsed_xml_wiktor = et.parse(xml_wiktor)

In [38]:
url_list = []
text_list = []
toponyms_list = []

for page in parsed_xml_wiktor.getroot():
    url = page.find('url')
    text = page.find('text')
    name = page.find('toponymName')
    wikipedia_name = page.find('pageTitle')
    lat = page.find('lat')
    lon = page.find('lon')

    toponyms = page.find('toponymIndices')
    toponym_list = []
    for toponym in toponyms:
        start = toponym.find('start')
        end = toponym.find('end')        
        toponym_list.append({'name':name.text, 'wikipedia_name':wikipedia_name.text, 'start':start.text, 'end': end.text, 'lat': lat.text, 'lon': lon.text, 'page': url.text})    
    
    url_list.append(url.text)
    text_list.append(text.text)
    toponyms_list.append(toponym_list)
    

df_wiktor = pd.DataFrame({'url' :url_list, 'text': text_list, 'toponyms': toponyms_list})
df_wiktor.head()

,url,text,toponyms
0,http://en.wikipedia.org/wiki/Santa_Maria%2C_Ca...,Santa Maria is a city near the Southern Califo...,"[{'name': 'Santa Maria', 'wikipedia_name': 'Sa..."
1,http://en.wikipedia.org/wiki/Santa_Maria%2C_Bu...,Santa María (/ˈsæntə məˈriə/; Tagalog pronunci...,"[{'name': 'Santa Maria', 'wikipedia_name': 'Sa..."
2,http://en.wikipedia.org/wiki/Santa_Maria%2C_Il...,Santa Maria (Nalibnos a Santa Maria) is a thir...,"[{'name': 'Santa Maria', 'wikipedia_name': 'Sa..."
3,http://en.wikipedia.org/wiki/Santa_Maria%2C_Ro...,Santa Maria (formerly Imelda) is a fifth class...,"[{'name': 'Santa Maria', 'wikipedia_name': 'Sa..."
4,http://en.wikipedia.org/wiki/Santa_Cruz%2C_Chile,"Santa Cruz is a Chilean city and commune, loca...","[{'name': 'Santa Cruz', 'wikipedia_name': 'San..."


In [39]:
name_list = []
lat_list = []
lon_list = []
page_list = []
wikipedia_name_list = []

for page in parsed_xml_wiktor.getroot():
    url = page.find('url')
    name = page.find('toponymName')
    wikipedia_name = page.find('pageTitle')
    lat = page.find('lat')
    lon = page.find('lon')

    name_list.append(name.text)
    lat_list.append(lat.text)
    lon_list.append(lon.text)
    page_list.append(url.text)
    wikipedia_name_list.append(wikipedia_name.text)

df_wiktor_poi = pd.DataFrame({'name' :name_list, 'wikipedia_name':wikipedia_name_list, 'lat': lat_list, 'lon': lon_list, 'page': url_list})
df_wiktor_poi.head()

,name,wikipedia_name,lat,lon,page
0,Santa Maria,"Santa Maria, California",34.9514,-120.4333,http://en.wikipedia.org/wiki/Santa_Maria%2C_Ca...
1,Santa Maria,"Santa Maria, Bulacan",14.8208,120.9636,http://en.wikipedia.org/wiki/Santa_Maria%2C_Bu...
2,Santa Maria,"Santa Maria, Ilocos Sur",17.3585,120.4981,http://en.wikipedia.org/wiki/Santa_Maria%2C_Il...
3,Santa Maria,"Santa Maria, Romblon",12.4417,122.1042,http://en.wikipedia.org/wiki/Santa_Maria%2C_Ro...
4,Santa Cruz,"Santa Cruz, Chile",-34.6372,-71.3631,http://en.wikipedia.org/wiki/Santa_Cruz%2C_Chile


In [40]:
df_wiktor_poi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            5000 non-null   object
 1   wikipedia_name  5000 non-null   object
 2   lat             5000 non-null   object
 3   lon             5000 non-null   object
 4   page            5000 non-null   object
dtypes: object(5)
memory usage: 195.4+ KB


In [41]:
df_wiktor_poi = df_wiktor_poi.drop_duplicates()
df_wiktor_poi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            5000 non-null   object
 1   wikipedia_name  5000 non-null   object
 2   lat             5000 non-null   object
 3   lon             5000 non-null   object
 4   page            5000 non-null   object
dtypes: object(5)
memory usage: 234.4+ KB


In [42]:
mlg_wiktor = pd.read_csv(os.path.dirname(os.getcwd())+'\\data\\evaluation-corpora\\data-patches\\WikToR_patches.tsv', sep = '\t', header = None)
mlg_wiktor = mlg_wiktor.rename(columns = {0:'toponym',1: 'old_coord',2: 'new_coord'})
mlg_wiktor.head()

,toponym,old_coord,new_coord
0,"Aba, Abia","5.1167 , 7.3667","5.1166666666667 , 7.3666666666667"
1,Abak,"4.9833 , 7.7833","4.9833333333333 , 7.7833333333333"
2,"Abbotsford, British Columbia","49.0489 , -122.2903","49.054611111111 , -122.328"
3,"Abbotsford, New South Wales","-33.8522 , 151.1273","-33.8521 , 151.127"
4,"Abbotsford, Victoria","-37.803 , 145.002","-37.8 , 145.0"


In [43]:
mlg_wiktor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9803 entries, 0 to 9802
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   toponym    9803 non-null   object
 1   old_coord  9803 non-null   object
 2   new_coord  9803 non-null   object
dtypes: object(3)
memory usage: 229.9+ KB


In [44]:
mlg_wiktor['mlg_key'] = mlg_wiktor.apply(lambda x: calc_mlg_key(x['toponym'],x['old_coord']),axis = 1)
mlg_wiktor.head()

,toponym,old_coord,new_coord,mlg_key
0,"Aba, Abia","5.1167 , 7.3667","5.1166666666667 , 7.3666666666667","Aba, Abia 5.1167,7.3667"
1,Abak,"4.9833 , 7.7833","4.9833333333333 , 7.7833333333333","Abak 4.9833,7.7833"
2,"Abbotsford, British Columbia","49.0489 , -122.2903","49.054611111111 , -122.328","Abbotsford, British Columbia 49.0489,-122.2903"
3,"Abbotsford, New South Wales","-33.8522 , 151.1273","-33.8521 , 151.127","Abbotsford, New South Wales -33.8522,151.1273"
4,"Abbotsford, Victoria","-37.803 , 145.002","-37.8 , 145.0","Abbotsford, Victoria -37.803,145.002"


In [45]:
def calc_wiktor_key(wikipedia_name, lat, lon):
    wikipedia_name = wikipedia_name.strip(' ')
    if (lat != None) & (lon != None):
        lat = lat.strip(' ')
        lon = lon.strip(' ')
        return wikipedia_name+' '+lat+','+lon

In [46]:
df_wiktor_poi['wiktor_key'] = df_wiktor_poi.apply(lambda x: calc_wiktor_key(x['wikipedia_name'], x['lat'], x['lon']),axis = 1)
df_wiktor_poi.head()

,name,wikipedia_name,lat,lon,page,wiktor_key
0,Santa Maria,"Santa Maria, California",34.9514,-120.4333,http://en.wikipedia.org/wiki/Santa_Maria%2C_Ca...,"Santa Maria, California 34.9514,-120.4333"
1,Santa Maria,"Santa Maria, Bulacan",14.8208,120.9636,http://en.wikipedia.org/wiki/Santa_Maria%2C_Bu...,"Santa Maria, Bulacan 14.8208,120.9636"
2,Santa Maria,"Santa Maria, Ilocos Sur",17.3585,120.4981,http://en.wikipedia.org/wiki/Santa_Maria%2C_Il...,"Santa Maria, Ilocos Sur 17.3585,120.4981"
3,Santa Maria,"Santa Maria, Romblon",12.4417,122.1042,http://en.wikipedia.org/wiki/Santa_Maria%2C_Ro...,"Santa Maria, Romblon 12.4417,122.1042"
4,Santa Cruz,"Santa Cruz, Chile",-34.6372,-71.3631,http://en.wikipedia.org/wiki/Santa_Cruz%2C_Chile,"Santa Cruz, Chile -34.6372,-71.3631"


In [47]:
df_wiktor_poi_unified = pd.merge(df_wiktor_poi, mlg_wiktor, how = 'outer', left_on = 'wiktor_key', right_on = 'mlg_key')
df_wiktor_poi_unified.head()

,name,wikipedia_name,lat,lon,page,wiktor_key,toponym,old_coord,new_coord,mlg_key
0,Santa Maria,"Santa Maria, California",34.9514,-120.4333,http://en.wikipedia.org/wiki/Santa_Maria%2C_Ca...,"Santa Maria, California 34.9514,-120.4333","Santa Maria, California","34.9514 , -120.4333","34.951388888889 , -120.43333333333","Santa Maria, California 34.9514,-120.4333"
1,Santa Maria,"Santa Maria, Bulacan",14.8208,120.9636,http://en.wikipedia.org/wiki/Santa_Maria%2C_Bu...,"Santa Maria, Bulacan 14.8208,120.9636","Santa Maria, Bulacan","14.8208 , 120.9636","14.818333333333 , 120.95633333333","Santa Maria, Bulacan 14.8208,120.9636"
2,Santa Maria,"Santa Maria, Ilocos Sur",17.3585,120.4981,http://en.wikipedia.org/wiki/Santa_Maria%2C_Il...,"Santa Maria, Ilocos Sur 17.3585,120.4981","Santa Maria, Ilocos Sur","17.3585 , 120.4981","17.366666666667 , 120.48333333333","Santa Maria, Ilocos Sur 17.3585,120.4981"
3,Santa Maria,"Santa Maria, Romblon",12.4417,122.1042,http://en.wikipedia.org/wiki/Santa_Maria%2C_Ro...,"Santa Maria, Romblon 12.4417,122.1042","Santa Maria, Romblon","12.4417 , 122.1042","12.441667 , 122.104167","Santa Maria, Romblon 12.4417,122.1042"
4,Santa Cruz,"Santa Cruz, Chile",-34.6372,-71.3631,http://en.wikipedia.org/wiki/Santa_Cruz%2C_Chile,"Santa Cruz, Chile -34.6372,-71.3631","Santa Cruz, Chile","-34.6372 , -71.3631","-34.633333333333 , -71.366666666667","Santa Cruz, Chile -34.6372,-71.3631"


In [48]:
df_wiktor_poi_unified.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9896 entries, 0 to 9895
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            5000 non-null   object
 1   wikipedia_name  5000 non-null   object
 2   lat             5000 non-null   object
 3   lon             5000 non-null   object
 4   page            5000 non-null   object
 5   wiktor_key      5000 non-null   object
 6   toponym         9803 non-null   object
 7   old_coord       9803 non-null   object
 8   new_coord       9803 non-null   object
 9   mlg_key         9803 non-null   object
dtypes: object(10)
memory usage: 850.4+ KB


In [49]:
df_wiktor_poi_unified.to_csv(os.path.dirname(os.getcwd())+'\\data\\extracted-annotated-locations\\WikToR_poi.csv')

## GeoCorpora POI Extraction

In [50]:
df_geocorpora = pd.read_csv(os.path.dirname(os.getcwd())+'\\data\\evaluation-corpora\\original-datasets\\geocorpora.tsv', sep = '\t')
df_geocorpora.head()

,feature_id,tweet_id_str,tweet_text,tweet_created_at,tweet_user_location,tweet_user_timezone,tweet_coordinates_latitude,tweet_coordinates_longitude,tweet_placeid,char_position,...,toponym,country_code,longitude,latitude,surrogate_geojson,uncertain_semantics,vague,overlapping_ambiguous,non_overlapping_ambiguous,not_in_geonames
0,1,557916969433718785,Bird Flu: Rivers Government confirms outbreak ...,Wed Jan 21 15:05:45 +0000 2015,Nigeria,NaN,NaN,NaN,NaN,10.0,...,Rivers State,NG,6.82766,4.74974,NaN,True,NaN,NaN,NaN,NaN
1,2,557916969433718785,Bird Flu: Rivers Government confirms outbreak ...,Wed Jan 21 15:05:45 +0000 2015,Nigeria,NaN,NaN,NaN,NaN,85.0,...,Federal Republic of Nigeria,NG,8.00000,10.00000,NaN,NaN,NaN,NaN,NaN,NaN
2,3,557916969433718785,Bird Flu: Rivers Government confirms outbreak ...,Wed Jan 21 15:05:45 +0000 2015,Nigeria,NaN,NaN,NaN,NaN,99.0,...,Federal Republic of Nigeria,NG,8.00000,10.00000,NaN,NaN,NaN,NaN,NaN,NaN
3,4,559125629711708161,RT @JohnFugelsang: Lando Calrissian got a bett...,Sat Jan 24 23:08:32 +0000 2015,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,493108823859924992,Tosi oudot albumit -artikkelisarja alkaa! Ensi...,Sat Jul 26 19:01:19 +0000 2014,NaN,Helsinki,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
df_geocorpora_poi = df_geocorpora[['geoNameId','toponym','longitude','latitude']]

In [52]:
df_geocorpora_poi = df_geocorpora_poi.dropna()
df_geocorpora_poi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3100 entries, 0 to 7630
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   geoNameId  3100 non-null   float64
 1   toponym    3100 non-null   object 
 2   longitude  3100 non-null   float64
 3   latitude   3100 non-null   float64
dtypes: float64(3), object(1)
memory usage: 121.1+ KB


In [53]:
df_geocorpora_poi = df_geocorpora_poi.drop_duplicates()
df_geocorpora_poi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1012 entries, 0 to 7629
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   geoNameId  1012 non-null   float64
 1   toponym    1012 non-null   object 
 2   longitude  1012 non-null   float64
 3   latitude   1012 non-null   float64
dtypes: float64(3), object(1)
memory usage: 39.5+ KB


In [54]:
df_geocorpora_poi.to_csv((os.path.dirname(os.getcwd())+'\\data\\extracted-annotated-locations\\GeoCorpora_poi.csv'))